In [ ]:
!pip install -U sentence-transformers -q
!pip install git+https://github.com/Bots-Avatar/ExplainitAll -q

In [2]:
from explainitall.QA.interp_qa.KNNWithGenerative import FredStruct, PromptBot
from explainitall.QA.extractive_qa_sbert.SVDBert import SVDBertModel
from explainitall.QA.extractive_qa_sbert.QABotsBase import cos_dist
from sklearn.neighbors import KNeighborsClassifier
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
path = 'FractalGPT/SbertSVDDistil'
sbert = SentenceTransformer(path)
sbert[0].auto_model = SVDBertModel.from_pretrained(path)

In [ ]:
fred = FredStruct('SiberiaSoft/SiberianFredT5-instructor')

In [5]:
datasetQ = [
    'Привет!',
    'У меня нет имени, я простая вопросно-ответная система',
    'Мне 3 года',
    'Машина это средство передвижения',
    'Полупроводнико́вый трио́д — электронный компонент из полупроводникового материала, способный небольшим входным сигналом управлять значительным током в выходной цепи, что позволяет использовать его для усиления, генерирования, коммутации и преобразования электрических сигналов.',
    'В классическом понимании поезд представляет собой движущуюся по пути цепь из соединённых между собой «повозок» — единиц подвижного состава.',
    'С точки зрения научной систематики, домашняя кошка — млекопитающее семейства кошачьих отряда хищных. Одни исследователи рассматривают домашнюю кошку как подвид дикой кошки[5], другие — как отдельный биологический вид[6].',
]

In [6]:
# Только ответы
vects_x = sbert.encode(datasetQ)
m = vects_x.mean(axis=0)
s = vects_x.std(axis=0)
knn_vects_x = (vects_x - m)/s
knn = KNeighborsClassifier(metric=cos_dist)
knn.fit(knn_vects_x, datasetQ)

KNeighborsClassifier(metric=<function cos_dist at 0x7aded208cee0>)

In [7]:
bot = PromptBot(knn, sbert, fred, datasetQ, device='cpu')

In [11]:
bot.get_answers('Насколько ты старый?', top_k=4)

'Мне 3 года.'

In [13]:
bot.get_answers('Расскажи про машины', top_k=2)

'Привет! Машины - это транспортные средства, которые используются для перемещения людей и грузов по дорогам. Они могут быть как механическими, так и электрическими. Механические машины состоят из двигателя, коробки передач и других узлов. Электрические машины состоят только из электрического двигателя и электрического оборудования..'